In [1]:
# Initialize seed
from emobpy.tools import set_seed
set_seed()

------------------------------------------------
## Step 1: Vehicle mobility time series
------------------------------------------------
a) generation of a time series

------------------------------------------------

In [2]:
from emobpy import Mobility

In [3]:
m = Mobility(config_folder='config_files')

In [4]:
m.set_params(
             name_prefix="BEV1",
             total_hours=168, # one week
             time_step_in_hrs=0.25, # 15 minutes
             category="user_defined",
             reference_date="01/01/2020"
            )

In [5]:
m.set_stats(
            stat_ntrip_path="TripsPerDay.csv",
            stat_dest_path="DepartureDestinationTrip.csv",
            stat_km_duration_path="DistanceDurationTrip.csv",
            )

In [6]:
m.set_rules(rule_key="user_defined") # see /config_files/rules.yml, it contains a dictionary
                                     # whose key must be the same as rule_key. 
                                     # To see all possible rules `from emobpy.constants import RULE; RULE` 

In [7]:
m.run()

New profile running: BEV1_W2_1e3f9
Progress: 100% [7 / 7] days
Profile done: BEV1_W2_1e3f9
Elapsed time (min): 0.15


In [8]:
m.save_profile(folder="db", description='168 hrs 15 min step ref-date 01/01/2020')

 
See Log files
/Users/Jarusch/Documents/Hertie/Hertie School/OneDrive - Hertie School/Jarusch/Virtual_storage/Modelling/Emobpy/my_emobpy/log/emobpy.log


In [9]:
##m.profile

In [10]:
c.timeseries[c.timeseries.state != "home"] 

NameError: name 'c' is not defined

In [ ]:
c.profile[['consumption kWh/100 km', 'consumption kWh', 'battery discharge kWh']]

In [ ]:
ga.timeseries

In [ ]:
ga.profile

-------------------------
b) See some attributes

-------------------------

In [11]:
m.name

'BEV1_W2_1e3f9'

In [12]:
m.kind

'driving'

In [13]:
m.description

'168 hrs 15 min step ref-date 01/01/2020'

In [14]:
m.timeseries

,hh,state,distance
date,,,
2020-01-01 00:00:00,0,home,0
2020-01-01 00:15:00,0.25,home,0
2020-01-01 00:30:00,0.5,home,0
2020-01-01 00:45:00,0.75,home,0
2020-01-01 01:00:00,1,home,0
...,...,...,...
2020-01-07 22:45:00,166.75,home,0
2020-01-07 23:00:00,167,home,0
2020-01-07 23:15:00,167.25,home,0


In [15]:
m.profile.head()

,hr,state,departure,arrival,last_arrival,purpose,duration,weekday,category,distance,trip_duration
10.50,10.50,home,10.75,11.00,-3.00,shopping,13.75,Wednesday,user_defined,0.0,0.0
10.75,10.75,driving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,10.0
15.25,15.25,shopping,15.50,16.75,11.00,home,4.50,Wednesday,user_defined,0.0,0.0
16.50,16.50,driving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130.0,72.5
31.50,31.50,home,7.75,8.00,-7.25,shopping,15.00,Thursday,user_defined,0.0,0.0


-------------------------------
c) Profiles management

-------------------------------

In [16]:
from emobpy import DataBase

In [17]:
DB = DataBase('db')

In [18]:
DB.loadfiles()

In [19]:
DB.db.keys()

dict_keys(['BEV1_W2_1e3f9'])

In [20]:
DB.db[m.name]['user_rules']

{'weekday': {'n_trip_out': [1],
  'last_trip_to': {'home': True},
  'overall_min_time_at': {'home': 9},
  'min_state_duration': {'home': 0.25,
   'errands': 0.25,
   'escort': 0.25,
   'shopping': 0.25,
   'leisure': 0.25}},
 'weekend': {'n_trip_out': [1],
  'last_trip_to': {'home': True},
  'overall_min_time_at': {'home': 6},
  'min_state_duration': {'home': 0.25,
   'errands': 0.25,
   'escort': 0.25,
   'shopping': 0.25,
   'leisure': 0.25}}}

In [21]:
DB.db[m.name]['kind']

'driving'

-------------------------
d) Visualization

-------------------------

In [21]:
from emobpy.plot import NBplot

In [22]:
PLT = NBplot(DB)

In [23]:
fig_mobility = PLT.sgplot_dp(m.name)

In [24]:
fig_mobility

------------------------------------------------------
## Step 2: Driving consumption time series
------------------------------------------------------
a) Vehicle model configuration

------------------------------------------------------

In [22]:
from emobpy import Consumption, HeatInsulation, BEVspecs

In [23]:
DB.update()                # This load new files hosted in database folder as result of new generated files

In [24]:
# mname = list(DB.db.keys())[0]      # getting the id of the first mobility profile
mname = m.name

In [25]:
HI = HeatInsulation(True)            # Creating the heat insulation by copying the default configuration

In [26]:
BEVS = BEVspecs()                    # Database that contains BEV models

In [27]:
BEVS.show_models()

Audi
	e-tron 55 quattro
		2019
		2020
	e-tron 55 quattro Premium Plus
		2019
	e-tron 55 quattro Prestige
		2019
	e-tron Sportback 50 quattro
		2020
	e-tron Sportback 55 quattro
		2020
BMW
	i3 22 kWh
		2014
		2015
		2016
	i3 33 kWh
		2017
		2018
	i3 42 kWh
		2019
	i3s 33 kWh
		2018
	i3s 42 kWh
		2019
	i3s Edition RoadStyle 42 kWh
		2020
Chevrolet
	Bolt EV
		2017
		2018
		2019
		2020
	Spark EV
		2014
		2015
		2016
FIAT
	500e
		2013
		2016
		2019
Hyundai
	IONIQ Electric 28 kWh
		2019
	IONIQ Electric 38.3 kWh
		2020
	KONA Electric 64 kWh
		2019
Jaguar
	I-Pace
		2019
	I-Pace S EV400 AWD Automatic
		2020
KIA
	Soul EV
		2015
		2016
		2017
		2018
		2019
	Soul EV 39 kWh
		2020
	Soul EV 64 kWh
		2020
	e-Niro 39 kWh
		2019
	e-Niro 4
		2020
	e-Niro 64 kWh
		2019
MINI
	Cooper SE Level I
		2020
	Cooper SE Level II
		2020
	Cooper SE Level III
		2020
Nissan
	Leaf S
		2013
		2014
		2015
		2016
		2017
		2018
		2019
	Leaf S Plus
		2019
	Leaf SL
		2011
		2012
		2013
		2014
		2015
		2016
		2017
		2018
		20

In [28]:
BEVS.parameters

['acc_0_100_kmh',
 'axle_ratio',
 'battery_cap',
 'curb_weight',
 'drag_coeff',
 'motor_type',
 'height',
 'length',
 'market',
 'num_cells',
 'num_modules',
 'power',
 'reg_braking',
 'top_speed',
 'torque',
 'trunk_volume',
 'battery_type',
 'voltage',
 'weight',
 'width']

In [29]:
dataframe = BEVS.search_by_parameter()

Parameter: power
   brand                     model  year  value unit
0  Tesla             Model S P100D  2016  568.0   kw
1  Tesla  Model S Performance (SR)  2020  568.0   kw
2  Tesla  Model S Performance (AC)  2019  568.0   kw
3  Tesla   Model S P90DL Ludicrous  2016  568.0   kw
4  Tesla   Model S P90DL Ludicrous  2015  568.0   kw
5  Tesla              Model S P90D  2016  568.0   kw
6  Tesla              Model S P90D  2015  568.0   kw
7  Tesla   Model S P85DL Ludicrous  2015  568.0   kw
8  Tesla  Model S P85D 193kW+375kW  2015  568.0   kw
9  Tesla             Model S P100D  2018  568.0   kw


In [30]:
VW_ID3 = BEVS.model(('Volkswagen','ID.3',2020))    # Model instance that contains vehicle parameters

Fallback value 0.9 added for missing battery_charging_eff parameter.
Fallback value 0.95 added for missing battery_discharging_eff parameter.
Fallback value 0.95 added for missing transmission_eff parameter.
Fallback value 0.3 added for missing auxiliary_power parameter.
Fallback value 3.5 added for missing cabin_volume parameter.
Fallback value 1 added for missing hvac_cop_heating parameter.
Fallback value 2 added for missing hvac_cop_cooling parameter.


In [31]:
VW_ID3.parameters

{'acc_0_100_kmh': None,
 'axle_ratio': 10,
 'battery_cap': 45.0,
 'curb_weight': 1600.0,
 'drag_coeff': 0.267,
 'motor_type': 'Brushless DC',
 'height': 1.552,
 'length': 4.261,
 'market': 'Europe',
 'num_cells': None,
 'num_modules': None,
 'power': 93,
 'reg_braking': 'Yes',
 'top_speed': 160.0,
 'torque': 310,
 'trunk_volume': 0.39,
 'battery_type': None,
 'voltage': None,
 'weight': None,
 'width': 1.809,
 'Brand': 'Volkswagen',
 'EV Model': 'ID.3',
 'Model year': 2020,
 'pmr': 58.125,
 'inertial_mass': 463.99999999999994,
 'front_area': 2.807568,
 'battery_charging_eff': 0.9,
 'battery_discharging_eff': 0.95,
 'transmission_eff': 0.95,
 'auxiliary_power': 0.3,
 'cabin_volume': 3.5,
 'hvac_cop_heating': 1,
 'hvac_cop_cooling': 2}

----------------------------------------------------------------------
b) Calculate consumption for each trip and generate the time series

----------------------------------------------------------------------

In [32]:
c = Consumption(mname, VW_ID3)

In [33]:
mname = "BEV2_W1_10dfc"

In [34]:
c.load_setting_mobility(DB)

In [ ]:
c.run(
    heat_insulation=HI,
    weather_country='DE',
    weather_year=2016,
    passenger_mass=75,                   # kg
    passenger_sensible_heat=70,          # W
    passenger_nr=1.5,                    # Passengers per vehicle including driver
    air_cabin_heat_transfer_coef=20,     # W/(m2K). Interior walls
    air_flow = 0.02,                     # m3/s. Ventilation
    driving_cycle_type='WLTC',           # Two options "WLTC" or "EPA"
    road_type=0,                         # For rolling resistance, Zero represents a new road.
    road_slope=0
    )

In [ ]:
c.save_profile('db')

------------------------------
c) See some attributes

------------------------------

In [ ]:
c.name

In [ ]:
c.kind

In [ ]:
c.input

In [ ]:
c.brand, c.model, c.year

In [ ]:
c.timeseries # Consumption in kWh/timestep -> timestep 15 min in this example

In [ ]:
c.timeseries.consumption.max()

In [ ]:
c.profile.head()

-------------------------
d) Visualization

-------------------------

In [ ]:
fig_consumption = PLT.sankey(c.name, include=None, to_html=False, path=None)

In [ ]:
fig_consumption

-------------------------------------------------------------------
## Step 3: Grid availability time series
-------------------------------------------------------------------
a) probability distribution and power rating for charging stations

-------------------------------------------------------------------

In [ ]:
from emobpy import Availability

In [ ]:
DB.update()                               # This load new generated files that are hosted in database folder
cname = c.name                            # getting the id of the first consumption profile


station_distribution = {                  # Dictionary with charging stations type probability distribution per the purpose of the trip (location or destination)
    'prob_charging_point': {
        'errands': {'public': 0.5, 'none': 0.5},
        'escort': {'public': 0.5, 'none': 0.5},
        'leisure': {'public': 0.5, 'none': 0.5},
        'shopping': {'public': 0.5, 'none': 0.5},
        'home': {'public': 0.5, 'none': 0.5},
        'workplace': {'public': 0.0, 'workplace': 1.0, 'none': 0.0},   # If the vehicle is at the workplace, it will always find a charging station available (assumption)
        'driving': {'none': 0.99, 'fast75': 0.005, 'fast150': 0.005}}, # with the low probability given to fast charging is to ensure fast charging only for very long trips (assumption)
    'capacity_charging_point': {                                       # Nominal power rating of charging station in kW
        'public': 22,
        'home': 3.7,
        'workplace': 11,
        'none': 0,  # dummy station
        'fast75': 75,
        'fast150': 150}
}

--------------------------------
b) Generate time series

--------------------------------

In [ ]:
ga = Availability(cname, DB)

In [ ]:
ga.set_scenario(station_distribution)

In [ ]:
ga.run()

In [ ]:
ga.save_profile('db')

--------------------------------
b) See some attributes

--------------------------------

In [ ]:
ga.name

In [ ]:
ga.input

In [ ]:
ga.kind

In [ ]:
ga.battery_capacity

In [ ]:
ga.charging_eff

In [ ]:
ga.discharging_eff

In [ ]:
ga.soc_init

In [ ]:
ga.soc_min

In [ ]:
ga.timeseries

In [ ]:
consumption_ts = ga.timeseries.consumption ##
availability_ts = ga.timeseries.charging_cap

In [ ]:
availability_ts.to_pickle("availability_ts")
consumption_ts.to_pickle("consumption_ts")

-------------------------
c) Visualization

-------------------------

In [ ]:
fig_availability = PLT.sgplot_ga(ga.name, rng=None, to_html=False, path=None)

In [ ]:
fig_availability

----------------------------------------------------------
## Step 4: Grid electricity demand time series
----------------------------------------------------------
a) Selection of charging strategies

---------------------------------------

In [ ]:
from emobpy import Charging

In [ ]:
DB.update()

aname = ga.name                            # getting the id of the availability profile

strategies = [
              "immediate",                 # When battery has SOC < 100% then it charges immediatelly at a maximun power rating of the current charging station
              "balanced",                  # When battery has SOC < 100% then it charges immediatelly but at lower rating power to ensure 100% SOC at the end (before moving to another place).
              "from_0_to_24_at_home",      # Customized: starting time of charging (this case 0 hrs), final time of charging (this case 24 hrs), at could be one 'location' (this case 'home') or 'any'.
              "from_23_to_8_at_any"
             ]

------------------------------------------------------
b) generation of 4 grid electricity demand time series

------------------------------------------------------

In [ ]:
for option in strategies:
    ged = Charging(aname)
    ged.load_scenario(DB)
    ged.set_sub_scenario(option)
    ged.run()
    print(f'Creation Successful:{ged.success}')   # if False, modify the strategy to a less constrained.
    ged.save_profile('db')

-----------------------------------------------------------------------------
c) See some atributes for the last created time series as an example

-----------------------------------------------------------------------------

In [ ]:
ged.name

In [ ]:
ged.input

In [ ]:
ged.kind

In [ ]:
ged.option

In [ ]:
ged.timeseries

In [ ]:
ged.profile

-------------------------
d) Visualization

-------------------------

In [ ]:
fig_ged = PLT.sgplot_ged(ged.name, rng=None, to_html=False, path=None) # this looks for all strategies of a single grid availability time series, even though we provide one grid demand id

In [ ]:
fig_ged

------------------------------------------------------
### Visualize all time series of a vehicle profile
------------------------------------------------------

In [ ]:
fig_channel = PLT.overview(ged.name)

In [ ]:
fig_channel

------------------------------------------------------------
### Export all time-series in 'db' folder to [DIETER](https://diw-evu.gitlab.io/dieter_public/dieterpy/) format
-----------------------------------------------------------------------------------------------------------
After exporting see the two CSV files at "db" folder

--------------------------------------------------------------------

In [ ]:
from emobpy import Export

In [ ]:
DB.update()
Exp = Export()
Exp.loaddata(DB)
Exp.to_csv()
Exp.save_files()

-------------------------------------------------------------------------------
### Download weather data from [Zenodo](https://zenodo.org/record/1489915)
-------------------------------------------------------------------------------
Total size: 300 MB

This function allows us to select different countries and years when creating new driving consumption time-series.

After finishing the download, you will get the location of the files on your PC. If you want to add more countries, you can edit the CSV files there.

emobpy includes Germany's weather data only for 2016. This data set has several European countries and years from 2000-2017.

-------------------------------------------------------------------------------

In [ ]:
# from emobpy import Weather

In [ ]:
# WD = Weather()

In [ ]:
# WD.download_weather_data()